## Introduction to global ocean biogeochemical modelling with transport operators
To model marine biogeochemical processes on a global scale we need to be able to account for the movement of chemical constituents both horizontally and vertically. We do this with a <b><i>tracer transport operator</i></b>.  When this operator acts on a tracer field it produces the advective-diffusive divergence of the tracer.

In order to represent the transport operator on a computer we have to discretize the tracer concentration field and the operator.  Once discretized the tracer field is represented as a vector and the operator is represented as a sparse matrix. (A sparse matrix behaves the same way as a regular matrix. The only difference is that in a sparse matrix the majority of the entries are zeros. These zeros are not stored explicitly to save computer memory making it possible to deal with fairly high resolution ocean models. 

Mathematically, the discretization converts an expression with partial derivatives into a matrix vector product:

$$\nabla \cdot \left[\mathbf{u}-\mathbf{K}\cdot\nabla \right]C \longrightarrow \mathbf{T}\mathbf{c}$$

where $\mathbf{T}$ is the flux divergence transport matrix and $\mathbf{c}$ is the tracer concentration vector. 

One can go a long way towards understanding what a tracer transport operator is by playing with a simple box model. We therefore introuce a simple box model before moving on to the <i>Ocean Circulation Inverse Model</i> (OCIM).


The simple box model we consider is embeded in a $2\times2\times2$ "shoebox". It has 5 <i>wet</i> boxes and 3 <i>dry</i> boxes.
<img src="boxmodel.png" width =800>

In [1]:
a = 6367e3   # Earth radius           (m)
A = 4*pi*a^2 # Earth surface area     (m²)
d = 3700     # ocean depth            (m)
V = 0.75*A*d # volume of ocean        (m³)
h = 200      # thickness of top layer (m)

dz = [h*ones(4,1);(d-h)*ones(4,1)] # grid box thicknesses       (m)
dV = (dz/d).*((V/4)*ones(8,1))     # grid box volumes           (m³)
dAz = dV./dz                       # area of face ⟂ to z axis   (m²)
dy = sqrt.(dAz)                    # north-south side length    (m)
dx = sqrt.(dAz)                    # east-west side length      (m)
dAx = dV./dy                       # area of face ⟂ to x axis   (m²)
dAy = dV./dx                       # area of face ⟂ to y axis   (m²)

msk = [1, 1, 1, 0, 1, 1, 0, 0]     # wet-dry mask wet=1 dry = 0 
iwet = findall(x -> x == 1, msk)        # index to wet gridboxes
idry = findall(x -> x == 0, msk)        # index to dry gridboxes
srf = [1, 1, 1, 0, 0]              # surface mask srface=1 bottom = 0
isrf = findall(x -> x == 1, srf) ;

The circulation consists of 
<ul>
    <li>a meridional overturning circulation flowing from box 1 to box 2 to box 6 to box 5 and back to box 1</li>
    <li>a zonal current in a reentrant channel from box 1 to box 3 and back to box 1</li>
    <li>vertical mixing representing deep convection between box 2 and box 6</li>
</ul>
    

In [2]:
using LinearAlgebra
using SparseArrays
TRdiv = spzeros(8,8)
# "Antarctic Circumpoloar Current"
acc = 100e6  # (m³/s)
TRdiv += sparse([1,1],[1,3],dV[1]\[-acc,acc],8,8)
TRdiv += sparse([3,3],[3,1],dV[3]\[-acc,acc],8,8)
# "Meridional Overturning Circulation"
moc = 15e6    # (m³/s)
TRdiv += sparse([1,1],[1,5],dV[1]\[-moc,moc],8,8)
TRdiv += sparse([2,2],[2,1],dV[2]\[-moc,moc],8,8)
TRdiv += sparse([5,5],[5,6],dV[5]\[-moc,moc],8,8)
TRdiv += sparse([6,6],[6,2],dV[6]\[-moc,moc],8,8)
# vertical mixing at "high northern latitudes"
q = 10e6      # (m³/s)
TRdiv += sparse([2,2],[2,6],dV[2]\[-q,q],8,8)
TRdiv += sparse([6,6],[6,2],dV[6]\[-q,q],8,8)
TRdiv = TRdiv[iwet,iwet]
Matrix(TRdiv)

5×5 Array{Float64,2}:
 -6.01987e-9   0.0          5.23467e-9   7.852e-10     0.0        
  7.852e-10   -1.30867e-9   0.0          0.0           5.23467e-10
  5.23467e-9   0.0         -5.23467e-9   0.0           0.0        
  0.0          0.0          0.0         -4.48686e-11   4.48686e-11
  0.0          7.4781e-11   0.0          0.0          -7.4781e-11 

## An idealized radiocarbon simulation
Radiocarbon, <sup>14</sup>C, is produced by cosmic rays in the lower stratosphere and upper troposphere. It quickly reacts with oxygen to produce <sup>14</sup>CO<sub>2</sub>, which is then mixed throughout the troposphere and enters the ocean through air-sea gas exchange. Because the halflife of radiocarbon is only 5730 years a significant amount of deday can occur before the dissolved inorganic radiocarbon (DI<sup>14</sup>C) can mix uniformally throughout the ocean. As such the <sup>14</sup>C serves as a tracer label for water that was recently in contact with the atmosphere. 

Here we will perform an idealized radiocarbon simulation in our model. In this model we prescribe the atmospheric concentration to 1 and model the air-sea gas exchange using a constant piston velocity of $\kappa = $50m/10years. For the radioactive decay we use a timescale of $\tau = 5730$years/$\log(2)$.

In [3]:
sec_per_year = 365*24*60*60; 
κ = 50 / (10 * sec_per_year)     # m/s
τ = 5730 * sec_per_year / log(2) # 1/s
κ, τ

(1.5854895991882294e-7, 2.6069684053828802e11)

In [4]:
# Example use of units in Julia
# Here is one step at a time
using Unitful, UnitfulAstro
κ2 = 50u"m" / 10u"yr" # m/yr
κ2 = upreferred(κ2) # m/s
κ2 = ustrip(κ2) # strip unit (LinearAlgebra functions needs Floats for now)
τ2 = 5730u"yr" / log(2) # yr
τ2 = upreferred(τ) # s
τ2 = ustrip(τ2)
κ2, τ2

(1.5844043907014474e-7, 2.6069684053828802e11)

In [5]:
# Example #2 of use of units in Julia
# Here is the same in fewer lines
using Unitful, UnitfulAstro
κ3 = ustrip(upreferred(50u"m" / 10u"yr"))
τ3 = ustrip(upreferred(5730u"yr" / log(2)))
κ3, τ3

(1.5844043907014474e-7, 2.6087540001810876e11)

In [6]:
M = TRdiv
M += -κ * spdiagm(0 => srf) / h # air-sea loss operator
M += spdiagm(0 => ones(5)) / τ  # radioactive decay loss operator
s = κ * srf / h               # air-sea source rate
M, s

(
  [1, 1]  =  -6.80878e-9
  [2, 1]  =  7.852e-10
  [3, 1]  =  5.23467e-9
  [2, 2]  =  -2.09758e-9
  [5, 2]  =  7.4781e-11
  [1, 3]  =  5.23467e-9
  [3, 3]  =  -6.02358e-9
  [1, 4]  =  7.852e-10
  [4, 4]  =  -4.10327e-11
  [2, 5]  =  5.23467e-10
  [4, 5]  =  4.48686e-11
  [5, 5]  =  -7.09451e-11, [7.92745e-10, 7.92745e-10, 7.92745e-10, 0.0, 0.0])

In [7]:
# Using `sparse(Diagonal)` and `I` (for identity) is cleaner in my opinion:
M2 = TRdiv
M2 += -κ * sparse(Diagonal(srf)) / h # air-sea loss operator
M2 += I / τ                          # radioactive decay loss operator
M2 == M     # test that it's the same as yours

true

In [8]:
R = - M \ s

5-element Array{Float64,1}:
 1.0810074624789625
 1.06193910398955  
 1.0710347112785237
 1.2239973332582768
 1.1193562263922334

In [9]:
age = τ * log.(R) / sec_per_year

5-element Array{Float64,1}:
  643.9172442117496
  496.7969498748243
  567.3000097505122
 1670.8703768740365
  932.0947213381232

## An idealized simulation of a radioactive tracer with an atmospheric origin usig OCIM

In [10]:
using AIBECS
t = empty_parameter_table()               # initialize table of parameters
add_parameter!(t, :τ, 5730u"yr" / log(2)) # add the radioactive decay e-folding timescale
add_parameter!(t, :κ, 50u"m" / 10u"yr")
initialize_Parameters_type(t)             # Generate the parameter table

┌ Warning: Package Flatten does not have Unitful in its dependencies:
│ - If you have Flatten checked out for development and have
│   added Unitful as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Flatten
│ Loading Unitful into Flatten from project dependency, future warnings for Flatten are suppressed.
└ @ nothing nothing:840


In [11]:
t

,symbol,value,unit,printunit,mean_obs,variance_obs,optimizable,description
,Symbol,Float64,Unitful…,Unitful…,Float64,Float64,Bool,String
1,τ,2.60875e11,s,yr,NaN,NaN,false,
2,κ,1.5844e-7,m s^-1,m yr^-1,NaN,NaN,false,


In [12]:
# I added this cell for you to see how it prints nicely with the "print" units
# The table `t` prints the value in the SI units, 
# while the parameter p₀ prints in the units you used to create it
p₀

     τ = 8.27e+03 [yr] (fixed)
     κ = 5.00e+00 [m yr⁻¹] (fixed)


AIBECS.Parameters{Float64}


In [13]:
const mask, grd, T_OCIM = OCIM1.load() ;
T_radio(p) = T_OCIM

Loading OCIM1 with JLD2 ✅


T_radio (generic function with 1 method)

In [14]:
# I changed the syntax here, and refactored the non-explicit `constants` function
# into small, explicit functions:
const z = vector_of_depths(mask, grd)
const nb = number_of_wet_boxes(mask)
# I'd suggest putting this constant as a constant, outside of the source/sink functions
const dz1 = grd["dzt"][1]

36.1351139041634

In [15]:
function source_sink_radio(R, p)
    τ = p.τ
    κ = p.κ
    return κ * (z .< 20) .* (R .- 1.0) / dz1 - R ./ τ
end

source_sink_radio (generic function with 1 method)

In [16]:
# Other way of making source_sink_radio
# I think it may be clearer to define source and sink separately? Keep whichever you prefer
source_radio(R, p) = p.κ * (z .< 20) .* (R .- 1.0) / dz1
sink_radio(R, p) = R ./ p.τ
source_sink_radio2(R, p) = source_radio(R, p) - sink_radio(R, p)
R_test = rand(nb) # Random R for testing that they are the same below
source_sink_radio(R_test, p₀) == source_sink_radio2(R_test, p₀)

true

In [17]:
T_matrices = (T_radio,)           # bundles all the transport matrices in a tuple
sources_minus_sinks = (source_sink_radio,) # bundles all the source-sink functions in a tuple
F, ∇ₓF = state_function_and_Jacobian(T_matrices, sources_minus_sinks, nb) # generates the state function (and its Jacobian!)

(getfield(AIBECS, Symbol("#F#21")){getfield(AIBECS, Symbol("#T#17")){Tuple{typeof(T_radio)}},getfield(AIBECS, Symbol("#G#19")){Tuple{typeof(source_sink_radio)},getfield(AIBECS, Symbol("#tracers#15")){Int64,Int64}}}(getfield(AIBECS, Symbol("#T#17")){Tuple{typeof(T_radio)}}((T_radio,)), getfield(AIBECS, Symbol("#G#19")){Tuple{typeof(source_sink_radio)},getfield(AIBECS, Symbol("#tracers#15")){Int64,Int64}}((source_sink_radio,), getfield(AIBECS, Symbol("#tracers#15")){Int64,Int64}(200160, 1))), getfield(AIBECS, Symbol("#∇ₓF#23")){getfield(AIBECS, Symbol("#T#17")){Tuple{typeof(T_radio)}},getfield(AIBECS, Symbol("#∇ₓG#22")){Tuple{typeof(source_sink_radio)},Int64,Int64}}(getfield(AIBECS, Symbol("#T#17")){Tuple{typeof(T_radio)}}((T_radio,)), getfield(AIBECS, Symbol("#∇ₓG#22")){Tuple{typeof(source_sink_radio)},Int64,Int64}((source_sink_radio,), 200160, 1)))

In [18]:
x₀ = ones(nb)
prob = SteadyStateProblem(F, ∇ₓF, x₀, p₀)
R = solve(prob, CTKAlg())

u: 200160-element Array{Float64,1}:
 1.0782340705655866
 1.0773899567394316
 1.0765826199090471
 1.0766251534552782
 1.0788553006659538
 1.0797320452218182
 1.0789425060311295
 1.0764865293474566
 1.07501169638437  
 1.072839670175266 
 1.0676162475438296
 1.063549185551198 
 1.062416690002382 
 ⋮                 
 1.2112585903446846
 1.2127960311459196
 1.2111010078092417
 1.2116376572947276
 1.2126524324835084
 1.2118933652725452
 1.2120304467672054
 1.2120055175600286
 1.2144669516977566
 1.2129656455701916
 1.213860089631053 
 1.212721189573085 